# CAPP 30239 - Assignment 4 : Static Viz 2 (Theming & Geospatial Viz)

*Chicago Public Schools (SY 2018-2019) vs. Chicago Crimes*

LIZA MARIE SORIANO

## Setup

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
import sys
import pandas as pd
import numpy as np
import json
import altair as alt
from altair.expr import datum, if_
alt.renderers.enable('colab')

!pip install geopandas
import geopandas as gpd

  Using cached https://files.pythonhosted.org/packages/52/4f/6440a63c9367d981a91de458467eed4a8e259a26f24158071b610a1ed1dd/geopandas-0.6.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ea/55/61a5d274a210585b5d0c3dac81a82952a4baa7903e3642228d7a465fc340/Shapely-1.7.0-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/fa/36/cd777f88a92d069341b6cecdf38c2a56be06888ea4b6eabad104329ea8c7/pyproj-2.4.2.post1-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/be/04/31d0a6f03943b1684f32c9b861be40c1fd282468fa6bd54ddf4a774e6b0f/Fiona-1.8.13.tar.gz
    Complete output from command python setup.py egg_info:
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    
    ----------------------------------------


Command "python setup.py egg_info" failed with error code 1 in C:\Users\apex\AppData\Local\Temp\pip-install-63if3nrs\fiona\


ModuleNotFoundError: No module named 'geopandas'

## Data Used
- [Chicago Public Schools Profile Information for Schoolyear 2018-2019](https://data.cityofchicago.org/Education/Chicago-Public-Schools-School-Profile-Information-/kh4r-387c)
- [Chicago Public Schools Outcomes 2011-2019](https://cps.edu/SchoolData/Pages/SchoolData.aspx)
  - **High School Graduation/Dropouts**: Metrics --> Dropout and Graduation --> Cohort Dropout and Graduation Rates, 2015 Method --> School Level
  - **College Enrollment/Persistence**: Metrics --> College Enrollment and Persistence Data --> School Level
- [Chicago Crimes Data for Year 2018](https://data.cityofchicago.org/Public-Safety/Crimes-2018/3i3m-jwuy)
- [Chicago Community Area Boundaries](https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Community-Areas-current-/cauq-8yn6)
- [Census Tracts to Community Area Equivalence](http://robparal.blogspot.com/2012/04/census-tracts-in-chicago-community.html)

In [ ]:
CPS_1819 = "Chicago_Public_Schools_-_School_Profile_Information_SY1819"
CPS_5yrGrad = "5-Year_HS_Cohort_Rates"
CPS_4yrGrad = "4-Year_HS_Cohort_Rates"
CPS_College = "College_Enrollment_Persistence_2010-2018"

CRIMES_18 = "Crimes_-_2018"
COMM_AREAS = "Boundaries - Community Areas (current)"

## Helper Functions

In [ ]:
def path_to_data(file):
    #return '/content/drive/My Drive/Courses/CAPP 30239 - Data Viz for Policy Analysis/Data/{}'.format(file)
    return '..Data/{}'.format(file)

In [ ]:
def read_csv(filename, cols=None, col_types=None):
    '''
    Inputs:
        name of file (str)--must end in .csv
        columns to include from original (list)--if not specified, read all cols
        specifications of datatypes of cols (dict of {col : dtype})
    Outputs:
      '''
      df = pd.read_csv(path_to_data(filename), usecols=cols, dtype=col_types)
      return df

In [ ]:
def change_to_numeric(df, col_list):
    '''
    Change select columns to numeric datatype
    INPUTS: df, col_list (list of str) of attributes to change
    '''
    # Change select columns to NUMERIC type
    for col in col_list:
        df[col] = pd.to_numeric(df[col])

In [ ]:
def remove_trailing_character(df, col_list, character=None):
    '''
    Remove character or trailing whitespace from values in df columns.
    INPUT: df | col_list (list of str) of colnames from which to remove character/whitespace | char (str) to remove
    OUTPUT: df edited in place
    '''
    if character:
      for col in col_list:
        df[col] = df[col].str.rstrip(character)
    else:
      for col in col_list:
        df[col] = df[col].str.rstrip()

## Colors / Theme

In [ ]:
# Color Theme
blue = "#578ceb"
pink = "#ff5aaa"
yellow = "#ffb14e"
mint = "#99d8c9"
purple = "#7854b3"
teal = "#069695"
dark_pink = "#be176f"

# For Altair
#theme_range = [blue, pink, yellow, mint, purple, teal, dark_pink]
theme_range = [yellow, blue, pink, dark_pink, purple, teal, mint]
sequential_palette = ["#cfe8f3", 
                      "#a2d4ec", 
                      "#73bfe2", 
                      "#46abdb", 
                      "#1696d2", 
                      "#12719e"
                      ]

In [ ]:
# Adapted from https://towardsdatascience.com/consistently-beautiful-visualizations-with-altair-themes-c7f9f889602
def custom_theme():
    # Typography
    font = "Georgia"
    labelFont = "Palatino" 
    sourceFont = "Palatino"
    # Colors
    main_palette = theme_range
    sequential_palette = ["#cfe8f3", 
                          "#a2d4ec", 
                          "#73bfe2", 
                          "#46abdb", 
                          "#1696d2", 
                          "#12719e"
                          ]
    return {
        # width and height are configured outside the config dict because they are Chart configurations/properties not chart-elements' configurations/properties.
        "width": 685, # from the guide
        "height": 380, # not in the guide
        "config": {
            "title": {
                "fontSize": 18,
                "font": font,
                "anchor": "middle",
                "fontColor": teal
            },
            "range": {
                "category": main_palette,
                "diverging": sequential_palette,
            },
            "legend": {
                "labelFont": labelFont,
                "labelFontSize": 12,
                "symbolSize": 100, # default
                "titleFont": font,
                "titleFontSize": 12,
                "rowPadding": 4,
                "orient": "right",
                "offset": 15,
            },
              }
                }

In [ ]:
# Adapted from: https://github.com/altair-viz/altair/issues/268
alt.themes.register('my_theme', custom_theme)
alt.themes.enable('my_theme')

## CPS DATA

### Read, Explore, & Modify Data

#### Load CPS Profile Information
- ***cps_df***

In [ ]:
cps_df = read_csv(CPS_1819 + ".csv", col_types={'School_ID':str})

In [ ]:
''' ADD ATTRIBUTES '''
# Add School Year
cps_df['schoolyear'] = '2018-2019'

# Add % Demographics
dmg_perc_cols = {'Student_Count_Low_Income': '%low_inc', \
                 'Student_Count_Special_Ed': '%special_ed', \
                 'Student_Count_English_Learners': '%esl', \
                 'Student_Count_Black': '%black', \
                 'Student_Count_Hispanic': '%hisp', \
                 'Student_Count_White': '%white', \
                 'Student_Count_Asian': '%asian', \
                 'Student_Count_Native_American': '%native', \
                 'Student_Count_Other_Ethnicity': '%other', \
                 'Student_Count_Asian_Pacific_Islander': '%as.pacif', \
                 'Student_Count_Multi': '%multirace', \
                 'Student_Count_Hawaiian_Pacific_Islander': '%hw.pacif', \
                 'Student_Count_Ethnicity_Not_Available': '%na'}

for count, perc in dmg_perc_cols.items():
    cps_df[perc] = cps_df[count] / cps_df['Student_Count_Total']

# Add Majority Race column
cps_df.loc[(cps_df['%black'] > 0.5), 'majority_race'] = 'majority_black'
cps_df.loc[(cps_df['%hisp'] > 0.5), 'majority_race'] = 'majority_hisp'
cps_df.loc[(cps_df['%white'] > 0.5), 'majority_race'] = 'majority_white'
cps_df['majority_race'].fillna('no_majority', inplace = True)


In [ ]:
cps_df.head()

In [ ]:
# Why we need additional data to look at outcomes
print(sum(cps_df['Graduation_Rate_School'].isna()), "schools out of", cps_df.shape[0], "are missing graduation rates.")

#### Load CPS High School Graduation/Dropout Information
- ***grad_df***

Year in CPS_5yrGrad refers to cohort who were freshman 5 years before that year, e.g. [Rate_Grad_5yr_2019] is % of students who were 9th-graders in SY 2014-15 that graduated by SY 2018-19 (which theoretically would also include all students who graduated SY 2017-18, i.e. after 4 years).

Let's grab cohorts that would normally graduate SY 2018-19 as well as the cohort right before. (Thus, freshmen during 2014-15 and 2015-16.)

In [ ]:
# Define cols to upload and to change to numeric
grad_cohort_1516 = ['School_ID', 'Rate_Dropout_4yr_2019', 'Rate_Grad_4yr_2019']
grad_cohort_1415 = ['School_ID', 'Rate_Dropout_5yr_2019', 'Rate_Grad_5yr_2019']
grad_cols_numeric = ['Rate_Dropout_4yr_2019', 'Rate_Dropout_5yr_2019', 'Rate_Grad_4yr_2019', 'Rate_Grad_5yr_2019']

In [ ]:
# Read in data
grad_df_4yr = read_csv(CPS_4yrGrad + ".csv", cols=grad_cohort_1516, col_types={'School_ID':str})
grad_df_5yr = read_csv(CPS_5yrGrad + ".csv", cols=grad_cohort_1415, col_types={'School_ID':str})
grad_df = pd.merge(grad_df_4yr, grad_df_5yr, on="School_ID", how='outer')
grad_df.head()

In [ ]:
''' CLEAN UP DATA '''
# Strip out '%' from the object values of rates
remove_trailing_character(grad_df, grad_cols_numeric, '%')

# Replace blank entries with NaNs (from: https://stackoverflow.com/questions/13445241/replacing-blank-values-white-space-with-nan-in-pandas)
grad_df = grad_df.replace(r'^\s*$', np.nan, regex=True)

# Convert cols to numeric
change_to_numeric(grad_df, grad_cols_numeric)

# Change cols to percentages
for col in grad_cols_numeric:
  grad_df[col] = grad_df[col] / 100

In [ ]:
grad_df.head()

#### Merge CPS Profile and HS Graduation info together

In [ ]:
# Check shape of dataframes before merging together
print("HS grad rates:", grad_df.shape)
print("CPS profile info:", cps_df.shape)

In [ ]:
# Merge CPS Profile Info with High School Graduation Outcomes Data
cps_df = pd.merge(cps_df, grad_df, on="School_ID", how='left')

In [ ]:
# Check that new dataframe's shape makes sense
print("New cps_df:", cps_df.shape)
cps_df.head()

#### Aggregate Citywide Numbers
- ***aggs_df***

In [ ]:
# Mean Graduation Percentage of High Schools
grad_mean_5yr_2019 = cps_df.loc[cps_df['Primary_Category'] == 'HS', 'Rate_Grad_5yr_2019'].mean()
grad_mean_4yr_2019 = cps_df.loc[cps_df['Primary_Category'] == 'HS', 'Rate_Grad_4yr_2019'].mean()

print("5-year cohort:", grad_mean_5yr_2019)
print("4-year cohort:", grad_mean_4yr_2019)

In [ ]:
# Mean Dropout Percentage of High Schools
dropout_mean_5yr_2019 = cps_df.loc[cps_df['Primary_Category'] == 'HS', 'Rate_Dropout_5yr_2019'].mean()
dropout_mean_4yr_2019 = cps_df.loc[cps_df['Primary_Category'] == 'HS', 'Rate_Dropout_4yr_2019'].mean()

print("5-year cohort:", dropout_mean_5yr_2019)
print("4-year cohort:", dropout_mean_4yr_2019)

In [ ]:
# Create citywide means dataframe
city_aggs = {'schoolyear_ending': [2019, 2019, 2019, 2019, 2019, 2019],
         'group': ['4-yr_cohort', '4-yr_cohort', '4-yr_cohort', '5-yr_cohort', '5-yr_cohort', '5-yr_cohort'],
         'rate_description': ['grad_rate', 'dropout_rate', 'other_rate', 'grad_rate', 'dropout_rate', 'other_rate'],
         'city_mean': [grad_mean_4yr_2019, dropout_mean_4yr_2019, 1-(grad_mean_4yr_2019 + dropout_mean_4yr_2019),
                       grad_mean_5yr_2019, dropout_mean_5yr_2019, 1-(grad_mean_5yr_2019 + dropout_mean_5yr_2019)]}
aggs_df = pd.DataFrame(data=city_aggs, dtype=np.int32)
aggs_df

In [ ]:
# See Number of High Schools and Majority Race
print('Total High Schools:', len(cps_df.loc[cps_df['Is_High_School'] == True,]))
print('Majority White high schools:', cps_df.loc[(cps_df['majority_race'] == 'majority_white'), 'Is_High_School'].sum())
print('Majority Black high schools:', cps_df.loc[(cps_df['majority_race'] == 'majority_black'), 'Is_High_School'].sum())
print('Majority Hispanic high schools:', cps_df.loc[(cps_df['majority_race'] == 'majority_hisp'), 'Is_High_School'].sum())
print('No Majority Race high schools:', cps_df.loc[(cps_df['majority_race'] == 'no_majority'), 'Is_High_School'].sum())

In [ ]:
# Add Majority-Race Outcomes to aggregate df
# (4YR COHORT)
cohort_4yr = {'majority_black': 'majority_black_HS_4yr_cohort',
          'majority_hisp': 'majority_hisp_HS_4yr_cohort',
          'no_majority': 'no_majority_HS_4yr_cohort'}

for race, group in cohort_4yr.items():
  grad_mean = cps_df[cps_df['majority_race'] == race]['Rate_Grad_4yr_2019'].mean()
  dropout_mean = cps_df[cps_df['majority_race'] == race]['Rate_Dropout_4yr_2019'].mean()

  df = pd.DataFrame({"schoolyear_ending":[2019, 2019, 2019], 
                    "group":3*[group],
                    "rate_description":['grad_rate', 'dropout_rate', 'other_rate'],
                    "city_mean": [grad_mean, dropout_mean, (1-(grad_mean + dropout_mean))]
                     })
  aggs_df = aggs_df.append(df, ignore_index = True)

In [ ]:
# Add Majority-Race Outcomes to aggregate df
# (5YR COHORT)
cohort_5yr = {'majority_black': 'majority_black_HS_5yr_cohort',
          'majority_hisp': 'majority_hisp_HS_5yr_cohort',
          'no_majority': 'no_majority_HS_5yr_cohort'}

for race, group in cohort_5yr.items():
  grad_mean = cps_df[cps_df['majority_race'] == race]['Rate_Grad_5yr_2019'].mean()
  dropout_mean = cps_df[cps_df['majority_race'] == race]['Rate_Dropout_5yr_2019'].mean()

  df = pd.DataFrame({"schoolyear_ending":[2019, 2019, 2019], 
                    "group":3*[group],
                    "rate_description":['grad_rate', 'dropout_rate', 'other_rate'],
                    "city_mean": [grad_mean, dropout_mean, (1-(grad_mean + dropout_mean))]
                     })
  aggs_df = aggs_df.append(df, ignore_index = True)

In [ ]:
aggs_df

### CHARTS : Demographics

#### Code

In [ ]:
lowinc_v_white = alt.Chart(cps_df).mark_point().encode(
    x=alt.X('%low_inc', axis=alt.Axis(format='%', title='% Low Income')),
    y=alt.Y('%white', axis=alt.Axis(format='%', title='% White')),
    color='Rating_Status',
    shape='Rating_Status'
).transform_filter(
    datum.Rating_Status != 'NOT APPLICABLE'
)
#lowinc_v_white

In [ ]:
lowinc_v_hisp = alt.Chart(cps_df).mark_point().encode(
    x=alt.X('%low_inc', axis=alt.Axis(format='%', title='% Low Income')),
    y=alt.Y('%hisp', axis=alt.Axis(format='%', title='% Hispanic')),
    color='Rating_Status',
    shape='Rating_Status'
).transform_filter(
    datum.Rating_Status != 'NOT APPLICABLE'
)
#lowinc_v_hisp

In [ ]:
lowinc_v_black = alt.Chart(cps_df).mark_point().encode(
    x=alt.X('%low_inc', axis=alt.Axis(format='%', title='% Low Income')),
    y=alt.Y('%black', axis=alt.Axis(format='%', title='% Black')),
    color='Rating_Status',
    shape='Rating_Status'
).transform_filter(
    datum.Rating_Status != 'NOT APPLICABLE'
)
#lowinc_v_black

In [ ]:
inc_by_race = alt.hconcat(lowinc_v_white, lowinc_v_hisp, lowinc_v_black).resolve_scale(y='shared')

inc_by_race_rating = alt.vconcat(
).properties(
    title={
      "text": ["Schools not in good standing tend to be poor, and",
               "majority white schools are probably not majority low-income."], 
      "subtitle": ["Chicago Public Schools: % Low-Income vs. % Race by School Performance",
                   "[Source: City of Chicago Data Portal]"],
      "color": teal,
      "subtitleColor": purple,
    }
)
for rating_status in ['GOOD STANDING', 'INTENSIVE SUPPORT', 'PROVISIONAL SUPPORT']:
  inc_by_race_rating &= inc_by_race.transform_filter(datum.Rating_Status == rating_status)

#inc_by_race_rating

In [ ]:
lowinc_hisp_black = alt.Chart(cps_df).mark_point().encode(
    x=alt.X('%low_inc', axis=alt.Axis(format='%', title='% Low Income')),
    y=alt.Y('%hisp', axis=alt.Axis(format='%', title='% Hispanic')),
    color=alt.Color('%black:Q', title='% Black', bin=alt.Bin(maxbins=5), 
                    scale=alt.Scale(range=[mint, teal, purple, dark_pink, pink])),
    shape=alt.Shape('%black:O', title=None,bin=alt.Bin(maxbins=5), sort='descending')
).properties(
    title={
      "text": ["Large clusters of very poor schools are either all Hispanic or all Black"], 
      "subtitle": ["Chicago Public Schools Student Population:",
                   "% Low-Income vs. % Hispanic vs. %Black",
                   "[Source: City of Chicago Data Portal]"],
      "color": teal,
      "subtitleColor": purple
    }
).configure_title(
    subtitleFontSize=12,
)
#lowinc_hisp_black

#### Graphics

Each Chicago public school has information about student population
demographics. There is also a performance metric ('Rating Status') by which each school's educational attainment is assessed. **The following graphs indicate that underperforming schools tend to be very poor and most likely non-White.**

*Rating Status* : Schools rated as needing intensive or provisional support are largely those with 80-100% low-income students and none of them have a sizeable White student population.

*Race vs Income* : Fifty percent or less of students in majority-White schools come from low-income families (only two exceptions). The low-income trend for Hispanic students is almost the complete opposite. Majority Hispanic schools are very likely to be majority low-income; the trend for Black student populations, though less stark, is similar.

In [ ]:
inc_by_race_rating

*Segregation?*

Looking specifically at % Hispanic graphs, we can see two clusters of schools along the 100% low-income end: almost 0% or almost 100% Hispanic. In other words, there is a large number of completely poor schools that are either all Hispanic or zero Hispanic. You can see these clusters replicated in the % Black graphs.

I wondered if these clusters speak to the racial and socioeconomic segregation in the city of Chicago. Despite a number of mixed-race student populations, many poor Hispanic and Black students are probably relegated away in schools that reflect the racial makeup of their neighborhoods. *I therefore guessed that the cluster of poor schools that are ~0% Hispanic are necessarily ~100% Black.* 

The following graphic provides a view that explores this idea.

In [ ]:
lowinc_hisp_black

This graph provides another view of the all-or-nothing race composition in many of Chicago's schools. Looking at the largely poor schools (close to 100% low income), the cluster at the bottom, where ~0% Hispanic, seem to be mostly Black (light pink circles for >= 80% Black). Meanwhile the cluster at the top, where ~100% Hispanic, consists of schools that are 20% or less Black (mint diamonds). This shows a pattern of segregation in Chicago public schools, while simultaneously demonstrating the poor economic status of Hispanic and Black student populations.

### CHARTS: Graduation Rates

#### Code

In [ ]:
chart_grad5yr = alt.Chart(cps_df).mark_rect().encode(
    x=alt.X('Rate_Grad_5yr_2019', bin=alt.Bin(maxbins=30), axis=alt.Axis(format='%', title='Graduation Rate')),
    y='majority_race:N',
    color='count()'
).transform_filter(
    (datum.Is_High_School == True) & (datum.Rate_Grad_5yr_2019 != 0)
).properties(
    width=500,
    height=200,
    title={
      "text": ["Chicago Public Schools 5-yr Graduation Rates", "by Majority Race"], 
      "subtitle": ["Majority Black schools are falling behind in graduation rates.",
                   "[Source: City of Chicago Data Portal, CPS.edu School Data]"],
      "color": teal,
      "subtitleColor": purple
    }
)
'''
.configure_title(
    subtitleFontSize=12,
)
'''
'''
heatmap = alt.Chart(source).mark_rect().encode(
    alt.X('IMDB_Rating:Q', bin=True),
    alt.Y('Rotten_Tomatoes_Rating:Q', bin=True),
    alt.Color('count()', scale=alt.Scale(scheme='greenblue'))
)
'''

chart_grad5yr_pts = alt.Chart(cps_df).mark_circle(
    color='black',
    size=5,
).encode(
    x='Rate_Grad_5yr_2019',
    y='majority_race:N',
)

chart_grad5yr + chart_grad5yr_pts
#chart_grad5yr

In [ ]:
chart_drop5yr = alt.Chart(cps_df).mark_point(shape='triangle', size=50).encode(
    x=alt.X('Rate_Dropout_5yr_2019:Q', axis=alt.Axis(format='%', title='Dropout Rate')),
    y='majority_race:N',
    color='Rating_Status:N',
).transform_filter(
    (datum.Is_High_School == True) & (datum.Rating_Status != 'NOT APPLICABLE')
).properties(
    width=500,
    height=200,
    title={
      "text": ["Chicago Public Schools 5-yr Dropout Rates", "by Majority Race"], 
      "subtitle": ["CPS seems to correctly identify high-dropout schools to be in need of intensive support.", 
                   "Many of these schools are majority Black.",
                   "[Source: City of Chicago Data Portal, CPS.edu School Data]"],
      "color": teal,
      "subtitleColor": purple
    }
).configure_title(
    subtitleFontSize=12,
)
#chart_drop5yr

In [ ]:
alt.Chart(cps_df).mark_point().encode(
    alt.X(alt.repeat("column"), type='quantitative', axis=alt.Axis(format='%')),
    alt.Y(alt.repeat("row"), type='quantitative', axis=alt.Axis(format='%')),
    color='Rating_Status:N'
).properties(
    width=150,
    height=150
).repeat(
    row=['%low_inc'],
    column=['%white', '%hisp']
)

In [ ]:
chart_mean5yr = alt.Chart(aggs_df).mark_rect().encode(
    y=alt.Y('group:N', axis=alt.Axis(title='', labels=False)),
    x=alt.X('city_mean:Q', axis=alt.Axis(format='%', title='Citywide Average')),
    row='rate_description:N',
    color='group:N'
).transform_filter(
    alt.FieldOneOfPredicate(field='group', oneOf=['no_majority_HS_5yr_cohort',
                                                  'majority_black_HS_5yr_cohort',
                                                  'majority_hisp_HS_5yr_cohort'])
).transform_filter(
    {'not': alt.FieldOneOfPredicate(field='rate_description', oneOf=['other_rate'])}
).properties(
    width=300,
    height=100,
    title={
      "text": ["Chicago Public Schools 5-yr Mean Outcomes by Majority Race", "(Citywide Average Graduation and Dropout Rates)"], 
      "subtitle": ["Majority Black high schools on average have a much higher dropout rate.",
                   "[Source: City of Chicago Data Portal, CPS.edu School Data]"],
      "color": teal,
      "subtitleColor": purple
    }
).configure_title(
    subtitleFontSize=12,
)
#chart_mean5yr

#### Graphics

In [ ]:
chart_grad5yr

Although there are plenty of majority white elementary schools, there are only majority Black or majority Hispanic public high schools in Chicago (making up about 48% and 39% of high schools, respectively). This might be a symptom of students going into private schools or going into large selective enrollment / competitive high schools that draw from all parts of the city. This in turn may mean that majority-race high schools could be made up of mostly neighborhood schools, therefore once again revealing the racial segregation of the city.

Either way, it looks like majority black high schools are lagging behind in terms of percent of students in the same freshman cohort graduating by their 5th year of high school.

In [ ]:
chart_drop5yr

The drop out rates of Chicago's schools are generally too high for comfort, but it seems like CPS is correctly identifying schools with high dropouts to be in need of more support. However, there seems to be a lower threshold for this rating status with majority Black schools compared to majority Hispanic schools. This might be worrisome for majority Hispanic schools that may need more attention and resources.

In [ ]:
chart_mean5yr

Consistent with the previous two charts, we observe that the average majority black high school in Chicago has a much higher dropout rate and much lower graduation rate than other schools. Majority Hispanic schools are also at a disadvantage compared to no-race-majority high schools.

### CHARTS: (Trying Isotope Graph)

In [ ]:
person = (
    "M1.7 -1.7h-0.8c0.3 -0.2 0.6 -0.5 0.6 -0.9c0 -0.6 "
    "-0.4 -1 -1 -1c-0.6 0 -1 0.4 -1 1c0 0.4 0.2 0.7 0.6 "
    "0.9h-0.8c-0.4 0 -0.7 0.3 -0.7 0.6v1.9c0 0.3 0.3 0.6 "
    "0.6 0.6h0.2c0 0 0 0.1 0 0.1v1.9c0 0.3 0.2 0.6 0.3 "
    "0.6h1.3c0.2 0 0.3 -0.3 0.3 -0.6v-1.8c0 0 0 -0.1 0 "
    "-0.1h0.2c0.3 0 0.6 -0.3 0.6 -0.6v-2c0.2 -0.3 -0.1 "
    "-0.6 -0.4 -0.6z"
)

In [ ]:
'''
source = pd.DataFrame([
      {'country': 'Great Britain', 'animal': 'cattle'},
      {'country': 'Great Britain', 'animal': 'cattle'},
    ])

domains = ['person']
'''
shape_scale = alt.Scale(
    #domain=domains,
    range=['M1.7 -1.7h-0.8c0.3 -0.2 0.6 -0.5 0.6 -0.9c0 -0.6 -0.4 -1 -1 -1c-0.6 0 -1 0.4 -1 1c0 0.4 0.2 0.7 0.6 0.9h-0.8c-0.4 0 -0.7 0.3 -0.7 0.6v1.9c0 0.3 0.3 0.6 0.6 0.6h0.2c0 0 0 0.1 0 0.1v1.9c0 0.3 0.2 0.6 0.3 0.6h1.3c0.2 0 0.3 -0.3 0.3 -0.6v-1.8c0 0 0 -0.1 0 -0.1h0.2c0.3 0 0.6 -0.3 0.6 -0.6v-2c0.2 -0.3 -0.1 -0.6 -0.4 -0.6z']
)

#domains = []

color_scale = alt.Scale(
    #domain=domains,
    range=[mint, teal, blue, purple]
)

alt.Chart(aggs_df).mark_point(filled=True, opacity=1, size=100).encode(
    alt.X('city_mean*100:Q', axis=None),
    alt.Y('rate_description:N', axis=None),
    #alt.Row('country:N', header=alt.Header(title='')),
    #shape=alt.ShapeValue(person),
    alt.Shape('rate_description:N', legend=None, scale=shape_scale),
    alt.Color('rate_description:N', legend=None, scale=color_scale),
).transform_window(
    x='rank()'
    #groupby=['country', 'animal']
).properties(width=550, height=140)

In [ ]:
alt.Chart(cps_df).mark_text(size=45, baseline='middle').encode(
    alt.X('x:O', axis=None),
    alt.Y('animal:O', axis=None),
    alt.Row('country:N', header=alt.Header(title='')),
    alt.Text('emoji:N')
).transform_calculate(
    emoji="{'cattle': '🐄', 'pigs': '🐖', 'sheep': '🐏'}[datum.animal]"
).transform_window(
    x='rank()',
    groupby=['country', 'animal']
).properties(width=550, height=140)

## CRIME DATA

### Read, Explore, Modify Data
- ***crime_df***

In [ ]:
# Read/Load
crime_df = read_csv(CRIMES_18 + ".csv", col_types={'ID':str, 'Community Area':str, 'Beat':str, 'District':str, 'Ward':str})

In [ ]:
# Explore
print('SHAPE:', crime_df.shape)
print('COLUMNS:', crime_df.columns)
print('DATATYPES:', crime_df.dtypes)
print('UNIQUE VALUES - Offense Types:', crime_df['Primary Type'].nunique())
print('UNIQUE VALUES - Community Areas:', crime_df['Community Area'].nunique())

In [ ]:
''' ADD/MODIFY ATTRIBUTES '''
# Dates
crime_df['Date'] = pd.to_datetime(crime_df['Date'])
crime_df['Year'] = crime_df['Date'].dt.year
crime_df['Month'] = crime_df['Date'].dt.month

# Rename column
crime_df.rename(columns = {'Primary Type':'Offense Type'}, inplace = True)

### Crime by Neighborhood

#### Group Data by Community Area

- ***hood_crime_type df***
- ***hood_crimes df***

In [ ]:
# Get a df counting crimes by Type within each Community Area
hood_crime_type = crime_df.groupby(['Community Area','Offense Type'], as_index=False)[['Date']].count()

# Rename groupby/mini-df columns
hood_crime_type.rename(columns = {'Community Area':'community_area', 'Date':'crime_count', 'Offense Type': 'offense_type'}, inplace = True)

# Preview
hood_crime_type.head()

In [ ]:
# How many neighborhoods each crime type is seen
hood_crime_type['offense_type'].value_counts().head()

In [ ]:
# Get a df counting total crimes within each Community Area
hood_crimes = hood_crime_type.groupby(['community_area'], as_index=False)[['crime_count']].sum()

# Sort by crime_count
hood_crimes = hood_crimes.sort_values(by=['crime_count'])

# Preview
hood_crimes

In [ ]:
# Explore / Check Stats
# There are 2 crimes with NULL values for community area
print('SHAPE:', hood_crimes.shape)
print('TOTAL # OF CRIMES:', hood_crimes['crime_count'].sum())
print('MISSING VALUES FOR COMM_AREA:', crime_df['Community Area'].isna().sum())
print('HIGHEST # OF CRIMES IN ONE COMM_AREA:', hood_crimes['crime_count'].max())


#### Merge with Boundaries Info (for maps)

Following steps adapted from: https://www.districtdatalabs.com/altair-choropleth-viz

- ***comm_bounds gdf***
- ***crimes_gdf***

In [ ]:
# Load GeoJson of Community Area boundaries
comm_bounds = gpd.read_file(path_to_data(COMM_AREAS) + '.geojson')

print('DATAFRAME TYPE:', type(comm_bounds))
comm_bounds.head()

In [ ]:
# Merge crimes data with area boundaries data
crimes_gdf = comm_bounds.merge(hood_crimes, left_on='area_num_1', right_on='community_area', how='inner')

print('DATAFRAME TYPE:', type(crimes_gdf))
print('DATFRAME SHAPE:', crimes_gdf.shape)
crimes_gdf.head()

In [ ]:
# Determine center of each polygon and add as columns
crimes_gdf['centroid_lon'] = crimes_gdf['geometry'].centroid.x
crimes_gdf['centroid_lat'] = crimes_gdf['geometry'].centroid.y
crimes_gdf.head()

### CHARTS: Chloropleths

#### Prep Data to be Used for Altair

Following steps adapted from: https://www.districtdatalabs.com/altair-choropleth-viz
- ***gen_geojson function***
- ***gen_map function***

In [ ]:
# Convert GeoPandas df back to GeoJson
def gen_geojson(geodataframe):
  ''' Converts GeoPandas dataframe back to GeoJson file that Altair can use for maps'''
  #choro_json = json.loads(crimes_gdf.to_json())
  #choro_data = alt.Data(values=choro_json['features'])
  data  = alt.InlineData(values = geodataframe.to_json(),
                                format = alt.DataFormat(property='features',
                                                        type='json'))
  return data

In [ ]:
# Generate map
def gen_map(geodata, color_column, title):
    '''Generates map with crime choropleth and community area labels'''
    # Add Base Layer
    base = alt.Chart(geodata, title = title).mark_geoshape(
        stroke='black',
        strokeWidth=1
    ).encode(
    ).properties(
        width=400,
        height=400
    )
    # Add Choropleth Layer
    choro = alt.Chart(geodata).mark_geoshape(
        stroke='black'
    ).encode(
        color=alt.Color(color_column, 
                  type='quantitative', 
                  #scale=alt.Scale(scheme='bluegreen'),
                  title = "Crime Count")
    )
    # Add Labels Layer
    labels = alt.Chart(geodata).mark_text(baseline='top'
     ).properties(
        width=400,
        height=400
     ).encode(
         longitude='properties.centroid_lon:Q',
         latitude='properties.centroid_lat:Q',
         text='properties.community_area:O',
         size=alt.value(8),
         opacity=alt.value(1)
     )

    return base + choro + labels

#### Maps

In [ ]:
choro_data = gen_geojson(crimes_gdf)
crime_2018_map = gen_map(geodata=choro_data, color_column='properties.crime_count', title='2018')
crime_2018_map

### CHARTS: Offense Types

In [ ]:
crime_df.head()

In [ ]:
# HEATMAP + POINTS
'''
type_month = alt.Chart(crime_df).mark_rect().encode(
    x='Month:T',
    y='count():Q',
    #color='count()'
).transform_filter(
    (datum.Is_High_School == True) & (datum.Rate_Grad_5yr_2019 != 0)
).properties(
    width=500,
    height=200,
    title={
      "text": ["Chicago Public Schools 5-yr Graduation Rates", "by Majority Race"], 
      "subtitle": ["Majority Black schools are falling behind in graduation rates.",
                   "[Source: City of Chicago Data Portal, CPS.edu School Data]"],
      "color": teal,
      "subtitleColor": purple
    }
).configure_title(
    subtitleFontSize=12,
)
type_month
'''

heatmap = alt.Chart(crime_df).mark_rect().encode(
    alt.X('Ward:N'),
    alt.Y('Month:T'),
    alt.Color('count()', scale=alt.Scale(scheme='greenblue'))
)

points = alt.Chart(crime_df).mark_circle(
    color='black',
    size=5,
).encode(
    x='Ward:N',
    y='Month:T',
)

alt.data_transformers.disable_max_rows()
heatmap + points

In [ ]:
#alt.data_transformers.enable('default', max_rows=None)
#alt.data_transformers.enable('csv')
alt.data_transformers.disable_max_rows()

# SMALL MULTIPLES
types = alt.Chart(crime_df).transform_filter(
    alt.datum.Month == 1
).mark_area().encode(
    x='date:T',
    y='count():Q',
    color='Offense Type:N',
    row=alt.Row('Offense Type:N', sort=['BATTERY', 'ASSAULT', 'ROBBERY', 'HOMICIDE'])
).properties(height=50, width=400)

#with alt.data_transformers.disable_max_rows():
#   types.display()
types

In [ ]:
hood_crimes.head()

In [ ]:
crime_df.shape

In [ ]:
alt.Chart(hood_crimes).mark_point().encode(
    x='community_area:N',
    y='crime_count:Q',
)